In [12]:
import pandas as pd
from faker import Faker
import random
import datetime
import psycopg2

# Inicializa o Faker
fake = Faker()

# Gerando dados fake

In [13]:
# lista_nomes_produtos = [
#     "Camiseta",
#     "Calça Jeans",
#     "Tênis",
#     "Relógio",
#     "Mochila",
#     "Celular",
#     "Notebook",
#     "Câmera",
#     "Fone de Ouvido",
#     "Smartwatch"
# ]

# lista_nomes_vendedores = [
#     "João Silva",
#     "Maria Oliveira",
#     "Pedro Santos",
#     "Ana Rodrigues",
#     "Luiz Almeida",
#     "Mariana Costa",
#     "Carlos Pereira",
#     "Laura Fernandes",
#     "Lucas Souza",
#     "Juliana Lima"
# ]


# # Função para gerar dados para a tabela Fato_Vendas
# def generate_sales_data(num_records):
#     sales_data = []
#     for i in range(num_records):
#         sale = {
#             'ID_Fato': i + 1,
#             'ID_Produto': random.randint(1, 10),
#             'ID_Cliente': random.randint(1, 10000),
#             'ID_Tempo': random.randint(1, 10000),
#             'Data_Venda': fake.date_between(start_date='-1y', end_date='today'),
#             'Quantidade_Vendida': random.randint(1, 100),
#             'Valor_Total': round(random.uniform(10, 10000), 2),
#             'Nome_vendedor': random.choice(lista_nomes_vendedores)
#         }
#         sales_data.append(sale)
#     return pd.DataFrame(sales_data)



# # Função para gerar dados para a tabela Dim_Produto
# def generate_product_data(num_produto):
#     product_data = []
#     for i in range(num_produto):
#         product = {
#             'ID_Produto': i + 1,
#             'Nome_Produto': random.choice(lista_nomes_produtos),
#             'Descricao': fake.sentence(),
#             'Preco': round(random.uniform(10, 10000), 2)
#         }
#         product_data.append(product)
#     return pd.DataFrame(product_data)



# # Função para gerar dados para a tabela Dim_Produto
# def generate_product_data(num_produto):
#     product_data = []
#     for i in range(num_produto):
#         product = {
#             'ID_Produto': i + 1,
#             'Nome_Produto': random.choice(lista_nomes_produtos),
#             'Descricao': fake.sentence(),
#             'Preco': round(random.uniform(10, 10000), 2)
#         }
#         product_data.append(product)
#     return pd.DataFrame(product_data)


# # Função para gerar dados para a tabela Dim_Cliente
# lista_nomes_clientes = [
#     fake.name() for j in range(100)
# ]


# def generate_customer_data(num_clientes):
#     customer_data = []
#     for i in range(num_clientes):
#         customer = {
#             'ID_Cliente': i + 1,
#             'Nome_Cliente': lista_nomes_clientes[i],
#             'Endereco': fake.address(),
#             'Contato': fake.phone_number()
#         }
#         customer_data.append(customer)
#     return pd.DataFrame(customer_data)


# # Função para gerar dados para a tabela Dim_Tempo
# def generate_time_data(num_records):
#     time_data = []
#     for i in range(num_records):
#         date = fake.date_between(start_date='-10y', end_date='today')
#         time_entry = {
#             'ID_Tempo': i + 1,
#             'Data': date.strftime('%Y-%m-%d'),
#             'Ano': date.year,
#             'Mes': date.month,
#             'Dia': date.day,
#             'Semana': date.isocalendar()[1]  # Semana do ano
#         }
#         time_data.append(time_entry)
#     return pd.DataFrame(time_data)



# # Número de registros para gerar
# num_records = 10000
# num_clientes = 100
# num_produto = 10

# # Gerar dados para cada tabela
# sales_df = generate_sales_data(num_records)
# product_df = generate_product_data(num_produto)
# customer_df = generate_customer_data(num_clientes)
# time_df = generate_time_data(num_records)


# #Gerar csv
# sales_df.to_csv('Vendas.csv', index=False, sep=';')
# product_df.to_csv('Produtos.csv', index=False, sep=';')
# customer_df.to_csv('Cliente.csv', index=False, sep=';')
# time_df.to_csv('Data_venda.csv', index=False, sep=';')


# Fazendo cosulta com python no banco

In [9]:
#ACESSANDO O BANCO DE DADOS POSTGRESQL
db_config = {
'host': 'localhost',
'database': 'postgres',
'user': 'postgres',
'password': '******',
}

In [24]:
#Criar View obter total de vendas por ano
conn = psycopg2.connect(**db_config)
cursor = conn.cursor()

vendas_ano = """
create or replace view total_vendas_ano as

SELECT 
    EXTRACT(year FROM to_date(data,'YYYY-MM-DD')) AS ano,
    SUM(cast(quantidade_vendida as numeric)) AS total_vendas
FROM 
    fato_vendas vd
JOIN 
    dim_tempo dt ON vd.id_tempo = dt.id_tempo
GROUP BY 
    EXTRACT(year FROM to_date(data,'YYYY-MM-DD'));
    
"""

cursor.execute(vendas_ano)
conn.commit()
conn.close()


In [34]:
#View total_vendas_ano


conn = psycopg2.connect(**db_config)
cursor = conn.cursor()

view_vendas_ano = """

select * from total_vendas_ano

"""

cursor.execute(view_vendas_ano)
vw_vendas_ano = cursor.fetchall()
conn.close()

df_vendas_ano = pd.DataFrame(vw_vendas_ano, columns=[desc[0] for desc in cursor.description])

df_vendas_ano

,ano,total_vendas
0,2024,13041
1,2016,51903
2,2021,49923
3,2018,45598
4,2015,53487
5,2014,35537
6,2020,49766
7,2017,51548
8,2023,51722
9,2022,53271


In [36]:
#Criar Dataframa lista de produtos com o total de vendas
conn = psycopg2.connect(**db_config)
cursor = conn.cursor()

sql_total_vendas_produto = """

select a.nome_produto,
	sum(cast(b.quantidade_vendida as numeric)) as total_vendas
from dim_produto a 
join fato_vendas b on a.id_produto = b.id_produto 
group by a.nome_produto 
order by total_vendas desc
    
"""

cursor.execute(sql_total_vendas_produto)
total_vendas_produto = cursor.fetchall()
conn.close()

df_total_vendas_produto = pd.DataFrame(total_vendas_produto, columns=[desc[0] for desc in cursor.description])

df_total_vendas_produto


,nome_produto,total_vendas
0,Smartwatch,146096
1,Notebook,102655
2,Câmera,53513
3,Relógio,53026
4,Fone de Ouvido,51521
5,Camiseta,50278
6,Calça Jeans,49167


In [35]:
#Criar Dataframe valor total de vendas por mês e ano, classificados pelo mês e ano:

conn = psycopg2.connect(**db_config)
cursor = conn.cursor()

valor_total_vendas_mes_ano = """

select 
    a.ano,
    a.mes, 
    sum(cast(quantidade_vendida as numeric)) AS quantidade_vendida
from 
    dim_tempo a 
join 
    fato_vendas b on  a.id_tempo = b.id_tempo 
group by 
    a.ano,
    a.mes
order by 
    cast(a.ano as numeric),
    cast(a.mes as numeric) desc

"""

cursor.execute(valor_total_vendas_mes_ano)
total_mes_ano = cursor.fetchall()
conn.close()

df_total_mes_ano = pd.DataFrame(total_mes_ano, columns=[desc[0] for desc in cursor.description])

df_total_mes_ano



,ano,mes,quantidade_vendida
0,2014,12,3978
1,2014,11,4877
2,2014,10,4283
3,2014,9,4219
4,2014,8,3060
...,...,...,...
116,2023,1,5003
117,2024,4,1210
118,2024,3,2847
119,2024,2,4151


In [30]:

#Criar uma view para informar produto, total de vendas, faturamento, media e meta de faturamento 9000000

conn = psycopg2.connect(**db_config)
cursor = conn.cursor()

sql_lucro = """

create or replace view produto_lucro as

select 
	b.nome_produto,
	sum(cast(a.quantidade_vendida as numeric)) as total_vendas,
	sum(cast(a.valor_total as numeric)) as faturamento,
	round(sum(cast(a.valor_total as numeric)) / sum(cast(a.quantidade_vendida as numeric)),2) as media,
	case
		when sum(a.valor_total::numeric) > 9000000 then 'Sim'
		else 'Nao'
	end as meta_concluida	
from 
	fato_vendas a
join
	dim_produto b on b.id_produto = a.id_produto 
group by 
	b.nome_produto
	
"""

cursor.execute(sql_lucro)
conn.commit()
conn.close()

In [33]:
#View produto_lucro

conn = psycopg2.connect(**db_config)
cursor = conn.cursor()

view_produto_lucro = """

select * from produto_lucro

"""

cursor.execute(view_produto_lucro)
produto_lucro = cursor.fetchall()
conn.close()

df_produto_lucro = pd.DataFrame(produto_lucro, columns=[desc[0] for desc in cursor.description])

df_produto_lucro

,nome_produto,total_vendas,faturamento,media,meta_concluida
0,Fone de Ouvido,51521,5048613.74,97.99,Nao
1,Smartwatch,146096,14808388.03,101.36,Sim
2,Câmera,53513,5368046.84,100.31,Nao
3,Notebook,102655,9816942.47,95.63,Sim
4,Camiseta,50278,5089960.24,101.24,Nao
5,Calça Jeans,49167,4825067.77,98.14,Nao
6,Relógio,53026,5168683.46,97.47,Nao
